# 5 Functions 

### 5.3 Augmented Assignment Statements

Create two names for the `str` object `123`, then from it create `1234`
and reassign (or rebind) one of the names:

In [ ]:
s1 = s2 = '123'
s1 is s2, s1, s2

In [ ]:
s2 = s2 + '4'
s1 is s2, s1, s2

  We can see this reassigns the second name so it is bound to a new
object.  This works similarly if we start with two names for one
`list` object and then reassign one of the names.

In [ ]:
m1 = m2 = [1, 2, 3]
m1 is m2, m1, m2

In [ ]:
m2 = m2 + [4]
m1 is m2, m1, m2

  If for the `str` objects we instead use an *augmented assignment
statement*, specifically *in-place add* **+=**, we get the same
behaviour.

In [ ]:
s1 = s2 = '123'

In [ ]:
s2 += '4'
s1 is s2, s1, s2

  However, for the `list` objects the behaviour changes.

In [ ]:
m1 = m2 = [1, 2, 3]

In [ ]:
m2 += [4]
m1 is m2, m1, m2

  The **+=** in **foo += 1** is not just syntactic sugar for **foo = foo +
1**.  **+=** and other augmented assignment statements have their
own bytecodes and methods.

  Let's look at the bytecode to confirm this.  Notice BINARY_ADD
vs. INPLACE_ADD.  Note the runtime types of the objects that `s` and
`v` are bound to is irrelevant to the bytecode that gets produced.

In [ ]:
import codeop, dis

In [ ]:
dis.dis(codeop.compile_command("a = a + b"))

In [ ]:
dis.dis(codeop.compile_command("a += b"))

In [ ]:
m2 = [1, 2, 3]

In [ ]:
m2

  Notice that `__iadd__` returns a value

In [ ]:
m2.__iadd__([4])

  and it also changed the list

In [ ]:
m2

In [ ]:
s2.__iadd__('4')


So what happened when `INPLACE_ADD` ran against the `str` object?

If `INPLACE_ADD` doesn't find `__iadd__` it instead calls `__add__` and
reassigns `s1`, i.e. it falls back to `__add__`.

https://docs.python.org/3/reference/datamodel.html#object.__iadd__:

> These methods are called to implement the augmented arithmetic
> assignments (+=, etc.). These methods should attempt to do the
> operation in-place (modifying self) and return the result (which
> could be, but does not have to be, self). If a specific method is
> not defined, the augmented assignment falls back to the normal
> methods.


  Here's similar behaviour with tuples, but a bit more surprising:

In [ ]:
t1 = (7,)
t1

In [ ]:
t1[0] += 1

In [ ]:
t1[0] = t1[0] + 1

In [ ]:
t1

In [ ]:
t2 = ([7],)
t2

In [ ]:
t2[0] += [8]

  What value do we expect t2 to have?

In [ ]:
t2

  Let's simulate the steps to see why this behaviour makes sense.

In [ ]:
m = [7]

In [ ]:
t2 = (m,)

In [ ]:
t2

In [ ]:
temp = m.__iadd__([8])

In [ ]:
temp == m

In [ ]:
temp is m

In [ ]:
temp

In [ ]:
t2

In [ ]:
t2[0] = temp

  For a similar explanation see https://docs.python.org/3/faq/programming.html#faq-augmented-assignment-tuple-error

### 5.4 Function Arguments are Passed by Assignment

Can functions modify the arguments passed in to them?

  When a caller passes an argument to a function, the function starts
execution with a local name (the parameter from its signature)
bound to the argument object passed in.

In [ ]:
def test_1a(s):
    print('Before:', s)
    s += ' two'
    print('After:', s)

In [ ]:
s1 = 'one'
s1

In [ ]:
test_1a(s1)

In [ ]:
s1

  To see more clearly why `s1` is still a name for 'one', consider
this version which is functionally equivalent but has two changes
highlighted in the comments:

In [ ]:
def test_1b(s):
    print('Before:', s)
    s = s + ' two'  # Changed from +=
    print('After:', s)

In [ ]:
test_1b('one')  # Changed from s1 to 'one'

  In both cases the name `s` at the beginning of `test_1a` and
`test_1b` was a name that was bound to the `str` object `'one'`,
and in both the function-local name `s` was re-bound to
the new `str` object `'one two'`.

  Let's try this with a `list`.

In [ ]:
def test_2a(m):
    print('Before:', m)
    m += [4]  # list += list is shorthand for list.extend(list)
    print('After:', m)

In [ ]:
m1 = [1, 2, 3]

In [ ]:
m1

In [ ]:
test_2a(m1)

In [ ]:
m1